<img src="https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Word2vect


In [107]:
import numpy as np
import math

In [108]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * (np.linalg.norm(b)))

### Datos

In [109]:
corpus = np.array(['que dia es hoy', 'martes el dia de hoy es martes', 'martes muchas gracias'])

Documento 1 --> que dia es hoy \
Documento 2 --> martes el dia de hoy es martes \
Documento 3 --> martes muchas gracias

### 1 - Obtener el vocabulario del corpus (los términos utilizados)
- Cada documento transformarlo en una lista de términos
- Armar un vector de términos no repetidos de todos los documentos

In [110]:
def vocabulary(corpus):
  vocabulary = set()
  for document in corpus:
    for word in document.split():
      vocabulary.add(word)

  return vocabulary

Verificamos la solución

In [111]:
vocabulary(corpus)

{'de', 'dia', 'el', 'es', 'gracias', 'hoy', 'martes', 'muchas', 'que'}

### 2- OneHot encoding
Data una lista de textos, devolver una matriz con la representación oneHotEncoding de estos

In [112]:
def one_hot_encoding(corpus):
  voc = list(vocabulary(corpus))
  fmatrix = np.zeros((len(corpus), len(voc)))

  for i, document in enumerate(corpus):
    for word in document.split():
      fmatrix[i][voc.index(word)] = 1

  return fmatrix.astype(int)

Verificamos la solución

In [113]:
one_hot_encoding(corpus)

array([[1, 0, 1, 0, 0, 0, 1, 0, 1],
       [1, 0, 0, 1, 1, 1, 1, 0, 1],
       [0, 1, 0, 0, 0, 1, 0, 1, 0]])

### 3- Vectores de frecuencia
Data una lista de textos, devolver una matriz con la representación de frecuencia de estos

In [114]:
def frequency_matrix(corpus):
  voc = list(vocabulary(corpus))
  fmatrix = np.zeros((len(corpus), len(voc)))

  for i, document in enumerate(corpus):
    for word in document.split():
      fmatrix[i][voc.index(word)] += 1

  return fmatrix.astype(int)

Verificamos la solución


In [115]:
frequency_matrix(corpus)

array([[1, 0, 1, 0, 0, 0, 1, 0, 1],
       [1, 0, 0, 1, 1, 2, 1, 0, 1],
       [0, 1, 0, 0, 0, 1, 0, 1, 0]])

### 4- TF-IDF
Data una lista de textos, devolver una matriz con la representacion TFIDF

In [116]:
def document_frequency(corpus, word):
  frequency = 0

  for i, document in enumerate(corpus):
    for w in document.split():
      if word == w:
        frequency += 1
        break

  return frequency

In [117]:
def tf_idf_matrix(corpus):
  voc = list(vocabulary(corpus))
  f_matrix = frequency_matrix(corpus)
  documents_count = len(corpus)
  matrix = np.zeros((documents_count, len(voc)))

  for i, document in enumerate(corpus):
    for word in document.split():
      index = voc.index(word)
      td = f_matrix[i][index]
      idf = math.log10(documents_count / document_frequency(corpus, word))
      matrix[i][index] = td * idf

  return matrix

Verificamos la solución

In [118]:
corpus

array(['que dia es hoy', 'martes el dia de hoy es martes',
       'martes muchas gracias'], dtype='<U30')

In [119]:
document_frequency(corpus, 'martes')

2

In [120]:
tf_idf_matrix(corpus)

array([[0.17609126, 0.        , 0.47712125, 0.        , 0.        ,
        0.        , 0.17609126, 0.        , 0.17609126],
       [0.17609126, 0.        , 0.        , 0.47712125, 0.47712125,
        0.35218252, 0.17609126, 0.        , 0.17609126],
       [0.        , 0.47712125, 0.        , 0.        , 0.        ,
        0.17609126, 0.        , 0.47712125, 0.        ]])

### 5 - Comparación de documentos
Realizar una funcion que reciba el corpus y el índice de un documento y devuelva los documentos ordenados por la similitud coseno

In [121]:
def compare(item1, item2):
    if item1['similarity'] > item2['similarity']:
        return -1
    elif item1['similarity'] < item2['similarity']:
        return 1
    else:
        return 0

In [122]:
import functools

In [123]:
def similar_documents_td_ifd(corpus, index):
  similarities = list()

  td_idf = tf_idf_matrix(corpus)
  document = td_idf[index]

  for i, d in enumerate(td_idf):
    if i == index:
      continue
    
    similarity = cosine_similarity(document, d)
    similarities.append({'similarity': similarity, 'document': corpus[i]})

  similarities.sort(key=functools.cmp_to_key(compare))
  return similarities

In [124]:
def similar_documents_frequency(corpus, index):
  similarities = list()

  matrix = frequency_matrix(corpus)
  document = matrix[index]

  for i, d in enumerate(matrix):
    if i == index:
      continue
    
    similarity = cosine_similarity(document, d)
    similarities.append({'similarity': similarity, 'document': corpus[i]})

  similarities.sort(key=functools.cmp_to_key(compare))
  return similarities

In [125]:
def similar_documents_one_hot(corpus, index):
  similarities = list()

  matrix = one_hot_encoding(corpus)
  document = matrix[index]

  for i, d in enumerate(matrix):
    if i == index:
      continue
    
    similarity = cosine_similarity(document, d)
    similarities.append({'similarity': similarity, 'document': corpus[i]})

  similarities.sort(key=functools.cmp_to_key(compare))
  return similarities

Verificamos las comparaciones

In [126]:
corpus

array(['que dia es hoy', 'martes el dia de hoy es martes',
       'martes muchas gracias'], dtype='<U30')

Usando TD-IDF

In [127]:
similar_documents_td_ifd(corpus, 2)

[{'similarity': 0.10845711727883083,
  'document': 'martes el dia de hoy es martes'},
 {'similarity': 0.0, 'document': 'que dia es hoy'}]

Usando matriz de frecuencias

In [128]:
similar_documents_frequency(corpus, 2)

[{'similarity': 0.3849001794597505,
  'document': 'martes el dia de hoy es martes'},
 {'similarity': 0.0, 'document': 'que dia es hoy'}]

Usando one hot encoding

In [129]:
similar_documents_one_hot(corpus, 2)

[{'similarity': 0.23570226039551587,
  'document': 'martes el dia de hoy es martes'},
 {'similarity': 0.0, 'document': 'que dia es hoy'}]

En conclusión vemos que tanto usando td-idf, matriz de frecuencias y one hot encoding se elije como mas similar al mismo documento, lo que tiene sentido ya que se refiere la afirmación del día actual, lo que difiere son los valores medidos para dicha similaridad, donde se calculó con una mayor similitud entre ambos vectores cuando se uso matriz de frecuencias.